In [ ]:
## GAN tensorflow 2.0 实现

import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# 生成器网络：将输入的 100 维的向量转换为 28x28x1 的图像
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_shape=(100,), activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(784, activation='tanh'),  # 输出图像大小为 28x28
        layers.Reshape((28, 28, 1))  # 转换形状为图像
    ])
    return model

# 判别器网络：将输入的 28x28x1 的图像转换为一个二分类结果
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # 输出一个判别结果，0表示伪造图像，1表示真实图像
    ])
    return model


# GAN 模型
def build_gan(generator, discriminator):
    # discriminator.trainable = False  # 冻结判别器的权重，只训练生成器
    discriminator.trainable = True  # 冻结判别器的权重，只训练生成器
    model = tf.keras.Sequential([generator, discriminator])
    return model

# 生成并保存生成器生成的图像。
def generate_and_save_images(generator, epoch, examples=10):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # 将图像归一化到[0, 1]范围
    
    plt.figure(figsize=(10, 1))
    for i in range(examples):
        plt.subplot(1, examples, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()


# 加载 MNIST 数据集
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # 归一化到[-1, 1]范围

# 创建生成器、判别器和 GAN 模型
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# 定义损失函数和优化器
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)


# 训练 GAN 模型

epochs = 10
batch_size = 128


for epoch in range(epochs):
    for _ in range(train_images.shape[0] // batch_size):
        # 随机选择一批真实图像
        real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]
        
        # 生成随机噪声作为输入
        noise = np.random.normal(0, 1, (batch_size, 100))
        
        # 使用生成器生成假图像
        generated_images = generator.predict(noise)
        
        # 训练判别器
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        
        with tf.GradientTape() as disc_tape:
            real_logits = discriminator(real_images)
            fake_logits = discriminator(generated_images)
            
            real_loss = cross_entropy(real_labels, real_logits)
            fake_loss = cross_entropy(fake_labels, fake_logits)
            total_discriminator_loss = real_loss + fake_loss
        
        gradients_discriminator = disc_tape.gradient(total_discriminator_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_discriminator, discriminator.trainable_variables))
        
        # 训练生成器
        with tf.GradientTape() as gen_tape:
            generated_images = generator(noise)
            fake_logits = discriminator(generated_images)
            generator_loss = cross_entropy(real_labels, fake_logits)
        
        gradients_generator = gen_tape.gradient(generator_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_generator, generator.trainable_variables))
        
    print(f'Epoch {epoch + 1}, Generator Loss: {generator_loss}, Discriminator Loss: {total_discriminator_loss}')
    if (epoch + 1) % 2 == 0:
        generate_and_save_images(generator, epoch + 1)


In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt

# 生成器网络：将输入的 100 维的向量转换为 28x28x1 的图像
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(256, input_shape=(100,), activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dense(784, activation='tanh'),  # 输出图像大小为 28x28
        layers.Reshape((28, 28, 1))  # 转换形状为图像
    ])
    return model

# 判别器网络：将输入的 28x28x1 的图像转换为一个二分类结果
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Flatten(input_shape=(28, 28, 1)),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # 输出一个判别结果，0表示伪造图像，1表示真实图像
    ])
    return model


# GAN 模型
def build_gan(generator, discriminator):
    # discriminator.trainable = False  # 冻结判别器的权重，只训练生成器
    discriminator.trainable = True  # 冻结判别器的权重，只训练生成器
    model = tf.keras.Sequential([generator, discriminator])
    return model

# 生成并保存生成器生成的图像。
def generate_and_save_images(generator, epoch, examples=10):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5  # 将图像归一化到[0, 1]范围
    
    plt.figure(figsize=(10, 1))
    for i in range(examples):
        plt.subplot(1, examples, i + 1)
        plt.imshow(generated_images[i, :, :, 0], cmap='gray')
        plt.axis('off')
    plt.savefig(f'gan_generated_image_epoch_{epoch}.png')
    plt.show()


In [2]:
# 加载 MNIST 数据集
(train_images, _), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # 归一化到[-1, 1]范围

# 创建生成器、判别器和 GAN 模型
generator = build_generator()
discriminator = build_discriminator()
gan = build_gan(generator, discriminator)

# 定义损失函数和优化器
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0002)


In [3]:
# 训练 GAN 模型

epochs = 10
batch_size = 128


for epoch in range(epochs):
    for _ in range(train_images.shape[0] // batch_size):
        # 随机选择一批真实图像
        real_images = train_images[np.random.randint(0, train_images.shape[0], batch_size)]
        
        # 生成随机噪声作为输入
        noise = np.random.normal(0, 1, (batch_size, 100))
        
        # 使用生成器生成假图像
        generated_images = generator.predict(noise)
        
        # 训练判别器
        real_labels = np.ones((batch_size, 1))
        fake_labels = np.zeros((batch_size, 1))
        
        with tf.GradientTape() as disc_tape:
            real_logits = discriminator(real_images)
            fake_logits = discriminator(generated_images)
            
            real_loss = cross_entropy(real_labels, real_logits)
            fake_loss = cross_entropy(fake_labels, fake_logits)
            total_discriminator_loss = real_loss + fake_loss
        
        gradients_discriminator = disc_tape.gradient(total_discriminator_loss, discriminator.trainable_variables)
        discriminator_optimizer.apply_gradients(zip(gradients_discriminator, discriminator.trainable_variables))
        
        # 训练生成器
        with tf.GradientTape() as gen_tape:
            generated_images = generator(noise)
            fake_logits = discriminator(generated_images)
            generator_loss = cross_entropy(real_labels, fake_logits)
        
        gradients_generator = gen_tape.gradient(generator_loss, generator.trainable_variables)
        generator_optimizer.apply_gradients(zip(gradients_generator, generator.trainable_variables))
        
    print(f'Epoch {epoch + 1}, Generator Loss: {generator_loss}, Discriminator Loss: {total_discriminator_loss}')
    if (epoch + 1) % 2 == 0:
        generate_and_save_images(generator, epoch + 1)


In [1]:
import torch 
from torch import nn
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:

def Gnet():
    net = nn.Sequential(
            nn.Linear(100, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, 784),
            nn.Tanh(),
            nn.BatchNorm1d(784),
            torch.nn.Unflatten(1, (28, 28, 1))
        )
    
    return net

def Dnet():
    net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 1),
        nn.Sigmoid()
    )

    return net


# 生成并保存生成器生成的图像。
def generate_and_save_images(generator, epoch, examples=10):
    noise = np.random.normal(0, 1, (examples, 100))
    generated_images = generator(torch.from_numpy(noise).float())
    generated_images = 0.5 * generated_images + 0.5  # 将图像归一化到[0, 1]范围

    plt.figure(figsize=(10, 1))
    for i in range(examples):
        plt.subplot(1, examples, i + 1)
        plt.imshow(generated_images[i, ...].detach().numpy(), cmap='gray')
        plt.axis('off')
    plt.savefig(f'gan_generated_image_epochaa_{epoch}.png')
    plt.show()


g_net = Gnet()
d_net = Dnet()

g_loss = nn.BCELoss()
d_loss = nn.BCELoss()

g_optimizer = torch.optim.Adam(g_net.parameters(), lr=0.0002)
d_optimizer = torch.optim.Adam(d_net.parameters(), lr=0.0002)


In [3]:
data = datasets.MNIST('./data', 
                      train=True, 
                      download=True, 
                      transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                        ]))

train_loader = torch.utils.data.DataLoader(data, batch_size=128, shuffle=True, drop_last=True)

In [ ]:
epochs = 10
batch_size = 128

for epoch in range(epochs):
    for i, (train_images, _) in enumerate(train_loader):
        # 使用生成器生成假图像
        noise = np.random.normal(0, 1, (batch_size, 100))
        g_fake = g_net(torch.from_numpy(noise).float())

        # 训练判别器
        d_optimizer.zero_grad()
        d_real = d_net(train_images)
        d_fake = d_net(g_fake)

        d_real_loss = d_loss(d_real, torch.ones_like(d_real))
        d_fake_loss = d_loss(d_fake, torch.zeros_like(d_fake))
        d_loss_all = d_real_loss + d_fake_loss
        d_loss_all.backward()
        d_optimizer.step()

        # 训练生成器
        g_optimizer.zero_grad()
        g_fake = g_net(torch.from_numpy(noise).float())
        d_fake = d_net(g_fake)
        g_loss_all = g_loss(d_fake, torch.ones_like(d_fake))
        g_loss_all.backward()
        g_optimizer.step()

    generate_and_save_images(g_net, epoch, examples=10)

In [49]:
## GAN pytorch 实现

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image
import os

data = datasets.MNIST('./data', 
                      train=True, 
                      download=True, 
                      transform=transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))
                        ]))

train_loader = torch.utils.data.DataLoader(data, batch_size=64, drop_last=True)

# Generator network
class Generator(nn.Module):
    def __init__(self, z_dim, image_dim):
        super(Generator, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.ReLU(),
            nn.BatchNorm1d(256),
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Linear(512, image_dim),
            nn.Tanh(),
            nn.Unflatten(1, (1, 28, 28))
        )

    def forward(self, x):
        return self.model(x)

# Discriminator network
class Discriminator(nn.Module):
    def __init__(self, image_dim):
        super(Discriminator, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(image_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

# Hyperparameters
z_dim = 100
image_dim = 28 * 28
batch_size = 64
lr = 0.0002
num_epochs = 10
sample_interval = 1

# Initialize generator and discriminator
generator = Generator(z_dim, image_dim)
discriminator = Discriminator(image_dim)

# Loss and optimizers
criterion = nn.BCELoss()
optimizer_G = optim.Adam(generator.parameters(), lr=lr)
optimizer_D = optim.Adam(discriminator.parameters(), lr=lr)

# Training loop
os.makedirs("images", exist_ok=True)

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Adversarial ground truths
        valid = torch.ones(batch_size, 1)
        fake = torch.zeros(batch_size, 1)

        # Train Generator
        optimizer_G.zero_grad()
        z = torch.randn(batch_size, z_dim)
        gen_images = generator(z)
        g_loss = criterion(discriminator(gen_images), valid)
        g_loss.backward()
        optimizer_G.step()

        # Train Discriminator
        optimizer_D.zero_grad()
        real_loss = criterion(discriminator(images), valid)
        fake_loss = criterion(discriminator(gen_images.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)
        d_loss.backward()
        optimizer_D.step()

        if i % 100 == 0:
            print(f"[Epoch {epoch}/{num_epochs}] [Batch {i}/{len(train_loader)}] [D loss: {d_loss.item()}] [G loss: {g_loss.item()}]")

        if epoch % sample_interval == 0:
            save_image(gen_images.data[:25], f"images/{epoch}.png", nrow=5, normalize=True)


[Epoch 6/10] [Batch 600/937] [D loss: 0.5478272438049316] [G loss: 1.5024466514587402]
[Epoch 6/10] [Batch 700/937] [D loss: 0.5461592674255371] [G loss: 1.8972598314285278]
[Epoch 6/10] [Batch 800/937] [D loss: 0.6142711639404297] [G loss: 1.1090264320373535]
[Epoch 6/10] [Batch 900/937] [D loss: 0.42470836639404297] [G loss: 2.109711170196533]
[Epoch 7/10] [Batch 0/937] [D loss: 0.6612293720245361] [G loss: 2.0073652267456055]
[Epoch 7/10] [Batch 100/937] [D loss: 0.44085943698883057] [G loss: 1.4523869752883911]
[Epoch 7/10] [Batch 200/937] [D loss: 0.445727676153183] [G loss: 1.2493127584457397]
[Epoch 7/10] [Batch 300/937] [D loss: 0.770003080368042] [G loss: 1.079391360282898]
[Epoch 7/10] [Batch 400/937] [D loss: 0.6184177398681641] [G loss: 1.5768896341323853]
[Epoch 7/10] [Batch 500/937] [D loss: 0.5827301144599915] [G loss: 1.5450780391693115]
[Epoch 7/10] [Batch 600/937] [D loss: 0.37065911293029785] [G loss: 1.642089605331421]
[Epoch 7/10] [Batch 700/937] [D loss: 0.6298384

In [7]:
## 使用imageio生成gif动图

import imageio.v3 as iio
import PIL
import numpy as np
import pathlib

# Display a single image using the epoch number
def display_image(epoch_no):
    return PIL.Image.open('./gan_generated_image_epoch_{}.png'.format(epoch_no))

anim_file = 'dcgan.gif'

images = pathlib.Path('./images').glob('*.png')
sorted_images = sorted(images)
images = np.stack([iio.imread(image) for image in sorted_images], axis=0)
iio.imwrite(anim_file, images, fps=1)


In [ ]:
## 使用imageio生成视频

import imageio.v3 as iio
import numpy as np

# read the video (it fits into memory)
# Note: this will open the image twice. Check the docs (advanced usage) if
# this is an issue for your use-case
metadata = iio.immeta("imageio:cockatoo.mp4", exclude_applied=False)
frames = iio.imread("imageio:cockatoo.mp4", index=None)

# manually convert the video
# gray_frames = np.dot(frames, [0.2989, 0.5870, 0.1140])
# gray_frames = np.round(gray_frames).astype(np.uint8)

images = pathlib.Path('./images').glob('*.png')
sorted_images = sorted(images)
images = np.stack([iio.imread(image) for image in sorted_images], axis=0)


# write the video
iio.imwrite("cockatoo_gray.mp4", images, fps=metadata["fps"])

In [10]:
import torch
import numpy as np

cross_entropy_loss = torch.nn.CrossEntropyLoss(reduction='none')

In [25]:
out = np.array([[0.1, 0.2, 0.3, 0.4], [0.3, 0.2, 0.4, 0.1]])
out = np.array([[0.1, 0.1], [0.4, 0.4]])
out = torch.from_numpy(out).float()

cross_entropy_loss(out, torch.zeros(out.size(0), dtype=torch.long))

tensor([0.6931, 0.6931])

In [18]:
out.size(0)

2

In [27]:
-np.log(0.1/(0.1+0.1)) * 1/2 - np.log(0.1/(0.1+0.1)) * 1/2

0.6931471805599453

In [26]:
input1 = np.array([[0.1, 0.1], [0.4, 0.4]])
input2 = np.array([0, 0])

cross_entropy_loss(torch.from_numpy(input1).float(), torch.from_numpy(input2).long())

tensor([0.6931, 0.6931])